In [1]:
from vayu_gnn.dbx.dbx_config import dbx_helper

In [18]:
city_nodes = {}
city_nodes['Patna'] = dbx_helper.read_pickle(dbx_helper.clean_input_path, 'node_locations/Patna', 'nodes.pickle')
city_nodes['Gurugram'] = dbx_helper.read_pickle(dbx_helper.clean_input_path, 'node_locations/Gurugram', 'nodes.pickle')

In [12]:
extra_nodes = {node:val for node, val in city_nodes['Patna'].items() if 'extra' in node}

In [13]:
lat, lon = extra_nodes['extra_node_1']['lat'], extra_nodes['extra_node_1']['long']

In [14]:
import requests, json

def get_pollution_data(lat, lon, start, end):
    # Enter your API key here
    api_key = "b9bf5cb8ec5f76b233c11a2e00a98f21"
    

    complete_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}"
    
    # get method of requests module
    # return response object
    response = requests.get(complete_url)
    
    # json method of response object 
    # convert json format data into
    # python format data
    x = response.json()
    
    return x


def get_pollution_forecast_data(lat, lon, start, end):
    # Enter your API key here
    api_key = "b9bf5cb8ec5f76b233c11a2e00a98f21"
    

    complete_url = f"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}"
    
    # get method of requests module
    # return response object
    response = requests.get(complete_url)
    
    # json method of response object 
    # convert json format data into
    # python format data
    x = response.json()
    
    return x


from datetime import datetime
import time

def year_month_to_unix(year_month_day: str) -> int:
    """Convert a 'YYYY-MM-DD' formatted string to a Unix timestamp (UTC)."""
    dt = datetime.strptime(year_month_day, "%Y-%m-%d")  # Convert to datetime
    return int(time.mktime(dt.timetuple()))  # Convert to Unix timestamp

may_unix = year_month_to_unix('2024-04-30')
mar_unix = year_month_to_unix('2025-03-01')

test = get_pollution_forecast_data(lat, lon, may_unix, mar_unix)


In [15]:
import pandas as pd

def air_quality_to_dataframe(data: list) -> pd.DataFrame:
    """
    Convert a list of air quality data dictionaries to a Pandas DataFrame.

    :param data: List of dictionaries containing air quality data
    :return: DataFrame with columns ['aqi', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'dt']
    """
    # Extract relevant fields
    df = pd.DataFrame([
        {
            'aqi': entry['main']['aqi'],
            'co': entry['components']['co'],
            'no': entry['components']['no'],
            'no2': entry['components']['no2'],
            'o3': entry['components']['o3'],
            'so2': entry['components']['so2'],
            'pm2_5': entry['components']['pm2_5'],
            'pm10': entry['components']['pm10'],
            'nh3': entry['components']['nh3'],
            'dt': entry['dt']
        }
        for entry in data
    ])

    return df

test_df = air_quality_to_dataframe(test['list'])

In [16]:
from datetime import datetime

def unix_to_readable(timestamp: int, fmt: str = "%Y-%m-%d %H:%M:%S", tz=None) -> str:
    """
    Convert a Unix timestamp to a human-readable datetime string.

    :param timestamp: Unix timestamp (seconds since epoch)
    :param fmt: Desired output format (default: "%Y-%m-%d %H:%M:%S")
    :param tz: Timezone (default: None, which means UTC)
    :return: Formatted datetime string
    """
    dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)
    return dt.strftime(fmt)

test_df['dt'] = test_df.dt.apply(unix_to_readable)

/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)


In [20]:
may_unix = year_month_to_unix('2024-04-30')
mar_unix = year_month_to_unix('2025-03-01')


cities = ['Patna', 'Gurugram']
for city in cities:
    list_node_dfs= []
    for node, coords in city_nodes[city].items():
        resulting_response = get_pollution_data(coords['lat'], coords['long'], may_unix, mar_unix)
        node_df = air_quality_to_dataframe(resulting_response['list'])
        node_df['dt'] = node_df.dt.apply(unix_to_readable)
        node_df = node_df.assign(node = node, lat = coords['lat'], long = coords['long'])
        list_node_dfs.append(node_df)
    dbx_helper.write_parquet(pd.concat(list_node_dfs), dbx_helper.clean_input_path, directory=f'pollution/{city}', filename='pollution.parquet') 

/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)
/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)
/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and schedu

Size of the Parquet file: 6.21 MB
File 'pollution.parquet' successfully uploaded to Dropbox path: '/input/clean/pollution/Patna/pollution.parquet'


/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)
/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp) if tz is None else datetime.fromtimestamp(timestamp, tz)
/var/folders/hs/1r2tlf751_l1nl8n9l2_wtpw0000gn/T/ipykernel_82328/2254394258.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and schedu

Size of the Parquet file: 5.96 MB
File 'pollution.parquet' successfully uploaded to Dropbox path: '/input/clean/pollution/Gurugram/pollution.parquet'
